## Deliverable 3. Create a Travel Itinerary Map.

In [117]:
import pandas as pd
import requests
import gmaps
import random

from config import g_key

gmaps.configure(api_key=g_key)

In [5]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
vacation_df = pd.read_csv("../Vacation_Search/WeatherPy_vacation.csv")
clean_hotel_df = vacation_df.dropna()
vacation_df.head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,3,Sao Miguel Do Araguaia,BR,74.64,broken clouds,-13.2750,-50.1628,Hotel Executivo Palace
1,13,Barabai,ID,78.85,overcast clouds,-2.5833,115.3833,HOTEL BOURAQ INDAH PERMAI
2,14,Rikitea,PF,75.56,moderate rain,-23.1203,-134.9692,Pension Maro'i
3,16,Thai Binh,VN,71.60,overcast clouds,20.4500,106.3333,Hội Liên Hiệp Phụ Nữ Tỉnh Thái Bình
4,19,Mar Del Plata,AR,71.01,clear sky,-38.0023,-57.5575,Gran Hotel Mar del Plata


In [6]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
# 3b. Get the latitude and longitude from each row and store in a new DataFrame.

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>
"""

hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]
locations = clean_hotel_df[["Lat", "Lng"]]

In [9]:
# 4a. Add a marker layer for each city to the map.
# 4b. Display the figure
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [203]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.

suggest_countries = clean_hotel_df["Country"].value_counts().where(suggest_countries_all > 3).dropna()
random_country = suggest_countries.sample(n=1)
sorted_cities = clean_hotel_df.loc[lambda clean_hotel_df: clean_hotel_df["Country"] == random_country.index[0]].sort_values(by=["Lat", "Lng"])
random_city = random.randint(0, len(sorted_cities)-4)

vacation_start = clean_hotel_df.loc[clean_hotel_df["City_ID"] == sorted_cities.iloc[random_city]["City_ID"]]
vacation_end = clean_hotel_df.loc[clean_hotel_df["City_ID"] == sorted_cities.iloc[random_city]["City_ID"]]
vacation_stop1 = clean_hotel_df.loc[clean_hotel_df["City_ID"] == sorted_cities.iloc[random_city + 1]["City_ID"]]
vacation_stop2 = clean_hotel_df.loc[clean_hotel_df["City_ID"] == sorted_cities.iloc[random_city + 2]["City_ID"]] 
vacation_stop3 = clean_hotel_df.loc[clean_hotel_df["City_ID"] == sorted_cities.iloc[random_city + 3]["City_ID"]]

In [204]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
start = vacation_start[["Lat", "Lng"]].to_numpy()
end = vacation_end[["Lat", "Lng"]].to_numpy()
stop1 = vacation_stop1[["Lat", "Lng"]].to_numpy()
stop2 = vacation_stop2[["Lat", "Lng"]].to_numpy()
stop3 = vacation_stop3[["Lat", "Lng"]].to_numpy()

In [211]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".

# origin  = str(start[0][0]) + "," + str(start[0][1])
# destination  = str(end[0][0]) + "," + str(end[0][1])
# wp1 = str(stop1[0][0]) + "," + str(stop1[0][1])
# wp2 = str(stop2[0][0]) + "," + str(stop2[0][1])
# wp3 = str(stop3[0][0]) + "," + str(stop3[0][1])
# wps = [wp1, wp2, wp3]
# base_url = "https://maps.googleapis.com/maps/api/directions/json"
# params = {
#     "origin": origin,
#     "destination": destination,
#     "waypoints": wps,
#     "mode": "driving",
#     "key": g_key
# }
# directions = requests.get(base_url, params=params).json()

origin  = (start[0][0], start[0][1])
destination  = (end[0][0], end[0][1])
wp1 = (stop1[0][0], stop1[0][1])
wp2 = (stop2[0][0], stop2[0][1])
wp3 = (stop3[0][0], stop3[0][1])
wps = [wp1, wp2, wp3]


fig = gmaps.figure()
route = gmaps.directions_layer(
        origin, destination, waypoints=wps,
        travel_mode='DRIVING')
fig.add_layer(route)
fig

Figure(layout=FigureLayout(height='420px'))

In [212]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([vacation_start, vacation_stop1, vacation_stop2, vacation_stop3],ignore_index=True)
itinerary_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,89,Mahebourg,MU,78.80,scattered clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
1,188,Bambous Virieux,MU,78.80,scattered clouds,-20.3428,57.7575,Casa Tia Villa
2,266,Quatre Cocos,MU,78.80,scattered clouds,-20.2078,57.7625,Emeraude Beach Attitude Hotel
3,453,Grand Gaube,MU,78.01,clear sky,-20.0064,57.6608,Veranda Paul et Virginie Hotel & Spa


In [215]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>
"""

# 10a Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]

In [216]:
# 11a. Add a marker layer for each city to the map.
# 11b. Display the figure
fig = gmaps.figure(center=(locations["Lat"][2], locations["Lng"][2]), zoom_level=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))